#  Hugging Face Transformers 微调训练

- 数据集下载
- 数据预处理
- 训练超参数设置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存



## 数据集下载

    数据集首页:[https://huggingface.co/datasets/Yelp/yelp_review_full](https://huggingface.co/datasets/Yelp/yelp_review_full)



In [6]:
from datasets import load_dataset
dataset_path = "/home/hengzq/workspace/modelscope/datasets/yelp_review_full"

dataset = load_dataset(dataset_path)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [8]:
dataset["train"][10]

{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [9]:
dataset["test"][10]

{'label': 1,
 'text': "Think Chuck E. Cheese for adults.  Skee Ball, video games pool tables.  Clean environment.  Good fun.\\n\\nUnfortunately, I went for a bite to eat and it was impossible to find anything good and healthy on the menu.  I ended up settling for spinach dip.  Sadly, they topped the dip off with horrible orange shredded cheese that appeared to have been popped in the microwave for a few seconds.  Blahhhh.  Trying to get something healthy, I ordered the apple pecan salad.  I swear the dressing came right out of the grocery store bottle.  I could barely eat the salad.  Too sweet.\\n\\nMy mom ordered a steak roll.....holy friedness!  The steak was more like hamburger fried with cheese and then stuffed into breading that was fried AGAIN!  Yowzer!  Artery clogger for sure.  \\n\\nI like the atmosphere.  I like the bar area.  Perhaps next time we'll just stop by for drinks instead."}

In [10]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [11]:
def show_random_elements(dataset, num_examples = 10):
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for col, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[col] = df[col].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(dataset["train"], 8)

,label,text
0,4 stars,"Everything you need in a Sportsbook. Big TVA, scoreboard and individual seating. Bar and food options close by, nice and convenient but not very busy which is what I prefer. If you like the crowds and loud excitement, not a place for you."
1,5 stars,"When I'm craving Tommy's , this is my spot . I grew up on this in LA and since moving to Vegas from Hawaii , finding Tommy's here has given me a piece of home and I only live down the street"
2,4 stars,"The two most important words to remember when going here are: Moscow mule.\n\nForget everything else; that's what you want. The jukebox has issues, as others have noted. The beer selection is decent, but there is far better in near proximity. The food is decent, but nothing to keep me coming back. What will keep me coming back, however, is the moscow mules. The first few I had there were made with cucumber vodka, and hot damn were they tasty! On another visit, I had a couple made with a vodka infused with mint and I can't even remember what else which one of the bartenders made herself.\n\nOh yes, speaking of the bartenders, they are awesome. Be nice to them, treat them well, and they will be awesome to you in return.\n\nSo in summary: Food, decent; Beer, decent; Moscow mules, delicious; Bartenders, awesome."
3,5 stars,"This is by far my favorite Mexican food in the valley. The tacos aren't massive, but what they lack in size they make up in taste. If you're going for the first time, get the number 5. It's a great way to try a burro, taco, and enchilada (I love the red sauce, my friend loves the green).\n\nWhenever I go camping, I pick up burros here and eat them with no food prep mess. My friends are always jealous of my food on those trips.\n\nI can't get enough of the chips and salsa. All around, I'm certain you'll love this place."
4,3 stars,"This one was the second five guys I've ever been to. The quality and taste were good, but I prefer the location on Lake Mead & Tenaya for cleanliness and friendliness."
5,2 star,"She's pretty nice, but got passive aggressive throughout the haircut. \nShe doesn't know how to cut male Asian hair, it was very mediocre. I was disappointed on multiple occasions hoping she would get it right. \nIt is convenient how you don't need to make an appointment, its 10 bucks, charges more for things like washing your hair if you have product before you come in."
6,5 stars,Quality service at an affordable price. Puddle Jumper goes out of their way to make the customer happy. Even after the giant dust storms last summer!
7,2 star,"I came in knowing I would have to either replace my transmission or get it rebuilt. \n\nWhen they did their free inspection, they said that they will need to do a more thorough inspection and charged about 280 (not exact but close to that amount). We didn't know the area so we decided to do the inspection to see what it could be. They later on tell us that it was just a simple solenoid. Which wouldn't cost as much as a replacement of a transmission or a rebuilt. \n\nWhen they lifted up the car and took the transmission apart, they told us that an motor mount was broken and we should get it fixed. (That meant more money) We got the motor mount fixed. \n\nAfter all that was done, the car was still having the same problem and nothing was really fixed. We ended up being out almost 400 and with still the same problem we came in there with. \n\nThey said that there is nothing else but to just either get it rebuilt or get a brand new transmission. Getting a rebuilt they would've charged me 2200. I just decided that was just way too much.\n\nI went to another place which was a lot cheaper than Paladin. And when I got my car fixed, the mechanic said that one of my solenoid was broken. (That would only mean that when Paladin was fixing my car, they broke my solenoid when they had it cleaned and put back into my car) That cost me more money to fix. =[ \n\nI have to admit though, the staff was very friendly and courteo

## 数据预处理

     下载数据集到本地后，使用Tokenizer来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

     Datasets的map方法，支持一次性在整个数据集上应用预处理函数。
     


In [13]:
#  使用填充到最大长度的策略，处理整个数据集：

from transformers import AutoTokenizer

model_path = "/home/hengzq/workspace/modelscope/models/bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_path)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation = True)

tokenized_datasets = dataset.map(tokenize_function, batched = True)

In [14]:
show_random_elements(tokenized_datasets["train"], num_examples = 2)

,label,text,input_ids,token_type_ids,attention_mask
0,5 stars,"Out of all the buffets I've been to in Vegas, this definitely takes the crown of the best right here.\n\nSure it costed $30 for a weekend lunch, but I definitely got my $30 worth and whole lot more. I was especially a fan of Pad Thai, the unlimited number of Spicy Tuna hand rolls, and Ribs! So delicious!\n\nI'd make every effort to come back here the next time I'm in town. Definitely worth it.","[101, 3929, 1104, 1155, 1103, 171, 9435, 6248, 146, 112, 1396, 1151, 1106, 1107, 6554, 117, 1142, 5397, 2274, 1103, 6371, 1104, 1103, 1436, 1268, 1303, 119, 165, 183, 165, 183, 1708, 3313, 1122, 2616, 1174, 109, 1476, 1111, 170, 5138, 5953, 117, 1133, 146, 5397, 1400, 1139, 109, 1476, 3869, 1105, 2006, 1974, 1167, 119, 146, 1108, 2108, 170, 5442, 1104, 19585, 1181, 6888, 117, 1103, 22921, 1295, 1104, 156, 20437, 1183, 17037, 1605, 1289, 12205, 117, 1105, 155, 13292, 1116, 106, 1573, 13108, 106, 165, 183, 165, 183, 2240, 112, 173, 1294, 1451, 3098, 1106, 1435, 1171, 1303, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
1,3 stars,"Had lunch at Gado Gado today with a large group of 20. We had our own section in the dining room. The place was nice on the inside and the restaurant was about half full. Service was friendly and efficient, especially for a large group.\n\nWe ordered 2 appetizers for the table. Rock shrimp and spinach artichoke dip. The Rick shrimp were breaded with a sweet sauce...very good! The spin dip was just ok. For my meal I ordered the veggie flatbread and a cup of their soup of the day, which was mushroom. Soup was lukewarm at best. Too bad, because the flavor was good. The flatbread had large fresh veggies on it with a crispy crust. My only complaint was that it had too much cheddar cheese on it. Wished it was just Italian cheeses.\n\nAll in all...A-OK!","[101, 6467, 5953, 1120, 144, 9359, 144, 9359, 2052, 1114, 170, 1415, 1372, 1104, 1406, 119, 1284, 1125, 1412, 1319, 2237, 1107, 1103, 7659, 1395, 119, 1109, 1282, 1108, 3505, 1113, 1103, 1656, 1105, 1103, 4382, 1108, 1164, 1544, 1554, 119, 2516, 1108, 4931, 1105, 7856, 117, 2108, 1111, 170, 1415, 1372, 119, 165, 183, 165, 183, 2924, 1162, 2802, 123, 12647, 26883, 26542, 1111, 1103, 1952, 119, 2977, 23982, 1105, 6898, 7291, 1893, 7255, 9143, 20866, 119, 1109, 5512, 23982, 1127, 8162, 1174, 1114, 170, 4105, 14313, 119, 119, 119, 1304, 1363, 106, 1109, 6898, 20866, 1108, 1198, 21534, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

`shuffle()`函数会随机重新排列列的值。

In [15]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed = 42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed = 42).select(range(1000))

## 加载模型


In [21]:
from transformers import AutoModelForSequenceClassification

model_path = "/home/hengzq/workspace/modelscope/models/bert-base-cased"

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/hengzq/workspace/modelscope/models/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 训练超参数设置

完整配置参数与默认值：[https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments](https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments)

源代码定义：[https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161](https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161)




In [22]:
from transformers import TrainingArguments

model_dir = "/home/hengzq/workspace/modelscope/models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，设置为100
training_args = TrainingArguments(output_dir = f"{model_dir}/test_trainer",
                                  logging_dir = f"{model_dir}/test_trainer/runs",
                                  logging_steps = 100)



ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [14]:
print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=no,
eval_use_gather_object=F

## 训练评估指标设置(Evaluate)

    Hugging Face Evaluate库 支持使用一行代码，获得数十种不同领域（自然语言处理，计算机视觉、强化学习等）的评估方法。

    完整的评估指标：[https://huggingface.co/evaluate-metric](https://huggingface.co/evaluate-metric)

    训练器（Trainer)在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告评估。

    Evaluate库提供了一个简单的准确率函数，你可以使用`evaluate.load`函数加载



In [5]:
#  定义度量函数
def compute_accuracy(predictions, references):
    correct = sum(p == r for p, r in zip(predictions, references))
    return correct / len(predictions) if len(predictions) > 0 else 0.0

### 训练过程指标监控
    通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在epoch结束时评估指标。

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir = f"{model_dir}/test_trainer",
                                  logging_dir = f"{model_dir}/test_trainer/runs",
                                  evaluation_strategy = "epoch",
                                  logging_steps = 100)

## 训练器基本介绍

xx


## 实战训练




In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics = compute_metrics
)

### 训练状态
- 使用`trainer.save_state`方法保存训练状态

In [ ]:
trainer.save_state()

## 模型保存

- 使用`trainer.save_model`方法保存模型，后续可以通过`from_pretrained()`方法重新加载


In [ ]:
trainer.save_model()